# Wow! Signal Audio Analysis

This notebook focuses specifically on the audio analysis of the Wow! signal.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, stats
import librosa
import librosa.display
import pywt
from IPython.display import display, Markdown, Audio
import seaborn as sns

# Try to import sounddevice, but continue if not available
try:
    import sounddevice as sd
    SOUNDDEVICE_AVAILABLE = True
except (ImportError, OSError):
    SOUNDDEVICE_AVAILABLE = False
    print("Warning: sounddevice module not available. Audio playback will be disabled.")

# Set some plotting parameters
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [14, 8]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 12

# Create a nice color palette
palette = sns.color_palette("viridis", 8)

# Define a function to get the project root directory
def get_project_root():
    """Get the absolute path to the project root directory."""
    return os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), ''))

# Define directories
project_root = get_project_root()
data_dir = os.path.join(project_root, 'data')
results_dir = os.path.join(project_root, 'results')

print("Setup complete!")

## 13. Audio Analysis of the Wow! Signal

For a unique perspective, we have an audio representation of the Wow! signal. Let's analyze its characteristics using audio processing techniques.

In [ ]:
# Path to the audio file
audio_path = os.path.join(data_dir, 'Wow_Signal_SETI_Project.mp3')

# Load and play the audio if it exists
if os.path.exists(audio_path):
    # Load the audio file
    y, sr = librosa.load(audio_path, sr=None)
    duration = librosa.get_duration(y=y, sr=sr)
    print(f"Audio loaded: {duration:.2f} seconds, {sr} Hz sample rate")
    
    # Play a sample of the audio (first 10 seconds)
    display(Audio(data=y[:sr*10], rate=sr))
else:
    print(f"Audio file not found at: {audio_path}")
    print("Please make sure the file exists or run the audio_analysis.py script first.")

## Visualizing the Audio Waveform and Spectrogram

Let's create visualizations of the audio signal to see its patterns and characteristics.

In [ ]:
# Create a function to check if audio is available before running analysis
def analyze_wow_audio():
    # Check if audio file was successfully loaded
    if 'y' not in globals():
        print("Audio data not available. Please load the audio file first.")
        return False
    return True

# If audio data is available, create visualizations
if os.path.exists(audio_path):
    # Create plots
    plt.figure(figsize=(14, 10))
    
    # Plot waveform
    plt.subplot(2, 1, 1)
    librosa.display.waveshow(y, sr=sr, alpha=0.8)
    plt.title('Wow! Signal Audio Waveform')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)
    
    # Plot spectrogram
    plt.subplot(2, 1, 2)
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Wow! Signal Spectrogram')
    plt.tight_layout()
    
    # Save the figure to the results directory
    plt.savefig(os.path.join(results_dir, 'wow_signal_audio_visualization.png'), dpi=300)
    plt.show()

## Spectral Analysis

Now let's analyze the frequency spectrum of the Wow! signal audio to identify dominant frequencies.

In [ ]:
# Perform spectral analysis if audio is available
if os.path.exists(audio_path) and analyze_wow_audio():
    # Compute the FFT
    n_fft = 4096
    fft_result = np.abs(librosa.stft(y, n_fft=n_fft))
    magnitude = np.mean(fft_result, axis=1)
    frequency = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    
    # Find peaks in the spectrum
    peaks, _ = signal.find_peaks(magnitude, height=np.mean(magnitude)*1.5, distance=20)
    peak_freqs = frequency[peaks]
    peak_mags = magnitude[peaks]
    
    # Sort peaks by magnitude
    peak_idx = np.argsort(peak_mags)[::-1][:10]  # Top 10 peaks
    top_peaks = [(peak_freqs[i], peak_mags[i]) for i in peak_idx]
    
    # Calculate spectral features
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
    spectral_flatness = librosa.feature.spectral_flatness(y=y)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    
    # Plot the spectrum
    plt.figure(figsize=(14, 7))
    plt.semilogy(frequency, magnitude)
    plt.plot(peak_freqs[peak_idx], peak_mags[peak_idx], 'ro', markersize=5)
    
    # Annotate the top 5 peaks
    for i, (freq, mag) in enumerate(top_peaks[:5]):
        plt.annotate(f"{freq:.1f} Hz", (freq, mag), 
                    xytext=(10, 10), textcoords='offset points',
                    fontsize=10, color='red')
    
    plt.title('Wow! Signal Audio Frequency Spectrum')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (log scale)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # Save the figure
    plt.savefig(os.path.join(results_dir, 'wow_signal_audio_spectrum.png'), dpi=300)
    plt.show()
    
    # Display key spectral features
    print(f"Mean Spectral Centroid: {np.mean(spectral_centroid):.2f} Hz")
    print(f"Mean Spectral Bandwidth: {np.mean(spectral_bandwidth):.2f} Hz")
    print(f"Mean Spectral Flatness: {np.mean(spectral_flatness):.4f} (0 = pure tone, 1 = white noise)")
    print(f"Mean Spectral Rolloff: {np.mean(spectral_rolloff):.2f} Hz")
    print("\nDominant Frequencies:")
    for i, (freq, mag) in enumerate(top_peaks[:5]):
        print(f"  {i+1}. {freq:.2f} Hz (magnitude: {mag:.2f})")

## Pattern Detection in Audio

Let's look for patterns or repeating elements in the audio signal that might indicate artificial origin.

In [ ]:
if os.path.exists(audio_path) and analyze_wow_audio():
    # Create a chromagram to detect musical patterns
    plt.figure(figsize=(14, 6))
    
    # We'll use a downsampled version to reduce memory usage
    y_downs = librosa.resample(y, orig_sr=sr, target_sr=sr//2)
    sr_downs = sr//2
    
    # Calculate chromagram (12 pitch classes)
    chroma = librosa.feature.chroma_stft(y=y_downs, sr=sr_downs)
    
    # Plot chromagram
    librosa.display.specshow(chroma, sr=sr_downs, x_axis='time', y_axis='chroma', cmap='coolwarm')
    plt.colorbar()
    plt.title('Chromagram: Pitch Class Distribution Over Time')
    plt.tight_layout()
    
    # Save the figure
    plt.savefig(os.path.join(results_dir, 'wow_signal_audio_chromagram.png'), dpi=300)
    plt.show()
    
    # Detect onsets (potential pattern starts)
    plt.figure(figsize=(14, 6))
    
    # Calculate onset strength
    onset_env = librosa.onset.onset_strength(y=y_downs, sr=sr_downs)
    times = librosa.times_like(onset_env, sr=sr_downs)
    
    # Find onset peaks
    onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr_downs)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr_downs)
    
    # Plot onset detection
    plt.plot(times, onset_env, label='Onset strength')
    plt.vlines(onset_times, 0, np.max(onset_env), color='r', alpha=0.7, linestyle='--', label='Onsets')
    plt.legend()
    plt.title('Onset Detection: Potential Signal Pattern Boundaries')
    plt.xlabel('Time (s)')
    plt.ylabel('Strength')
    plt.tight_layout()
    
    # Save the figure
    plt.savefig(os.path.join(results_dir, 'wow_signal_audio_onsets.png'), dpi=300)
    plt.show()
    
    # Display pattern statistics
    onset_intervals = np.diff(onset_times)
    
    if len(onset_intervals) > 0:
        print(f"Number of detected onsets: {len(onset_times)}")
        print(f"Mean interval between onsets: {np.mean(onset_intervals):.4f} seconds")
        print(f"Standard deviation of intervals: {np.std(onset_intervals):.4f} seconds")
        
        # Check for regularity (low standard deviation compared to mean indicates regular patterns)
        regularity = np.std(onset_intervals) / np.mean(onset_intervals) if np.mean(onset_intervals) > 0 else 0
        print(f"Pattern regularity coefficient: {regularity:.4f} (lower is more regular)")
        
        if regularity < 0.5:
            print("The signal shows significant regular patterning, suggesting possible artificial origin.")
        elif regularity < 0.7:
            print("The signal shows some regular patterns, but with natural variation.")
        else:
            print("No strong regular patterns detected in the signal.")
    else:
        print("No clear onsets detected in the signal.")

## Modulation Analysis

Let's analyze the signal for signs of modulation that might indicate an information-carrying signal.

In [ ]:
if os.path.exists(audio_path) and analyze_wow_audio():
    # We'll work with a downsampled signal
    y_downs = librosa.resample(y, orig_sr=sr, target_sr=sr//4)
    sr_downs = sr//4
    
    # Calculate amplitude envelope
    def get_envelope(y, frame_length=1024, hop_length=512):
        y_abs = np.abs(y)
        return librosa.util.frame(y_abs, frame_length=frame_length, hop_length=hop_length).max(axis=0)
    
    # Get amplitude and frequency modulation
    envelope = get_envelope(y_downs)
    
    # Create analytic signal using Hilbert transform
    analytic_signal = signal.hilbert(y_downs)
    amplitude_envelope = np.abs(analytic_signal)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = np.diff(instantaneous_phase) / (2.0*np.pi) * sr_downs
    
    # Plot modulations
    plt.figure(figsize=(14, 12))
    
    # Plot the original signal
    plt.subplot(3, 1, 1)
    time_downs = np.arange(len(y_downs)) / float(sr_downs)
    plt.plot(time_downs[:len(y_downs)], y_downs)
    plt.title('Original Signal')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    
    # Plot amplitude modulation
    plt.subplot(3, 1, 2)
    plt.plot(time_downs[:len(amplitude_envelope)], amplitude_envelope)
    plt.title('Amplitude Modulation')
    plt.xlabel('Time (s)')
    plt.ylabel('Envelope')
    
    # Plot frequency modulation
    plt.subplot(3, 1, 3)
    # Only plot middle section to avoid edge artifacts
    middle_start = int(len(instantaneous_frequency) * 0.1)
    middle_end = int(len(instantaneous_frequency) * 0.9)
    plt.plot(time_downs[middle_start:middle_end], 
             instantaneous_frequency[middle_start:middle_end])
    plt.title('Frequency Modulation')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'wow_signal_audio_modulation.png'), dpi=300)
    plt.show()
    
    # Analyze modulation characteristics
    # AM analysis
    am_var = np.var(amplitude_envelope)
    am_mean = np.mean(amplitude_envelope)
    am_modulation_index = np.sqrt(am_var) / am_mean if am_mean > 0 else 0
    
    # FM analysis
    fm_var = np.var(instantaneous_frequency[middle_start:middle_end])
    fm_mean = np.mean(instantaneous_frequency[middle_start:middle_end])
    fm_modulation_index = np.sqrt(fm_var) / fm_mean if fm_mean > 0 else 0
    
    print("Modulation Analysis:")
    print(f"AM Modulation Index: {am_modulation_index:.4f}")
    print(f"FM Modulation Index: {fm_modulation_index:.4f}")
    
    # Interpretation
    if am_modulation_index > 0.2 and fm_modulation_index < 0.1:
        print("Signal shows characteristics of amplitude modulation (AM)")
    elif fm_modulation_index > 0.2 and am_modulation_index < 0.1:
        print("Signal shows characteristics of frequency modulation (FM)")
    elif am_modulation_index > 0.2 and fm_modulation_index > 0.2:
        print("Signal shows characteristics of both AM and FM modulation")
    else:
        print("No strong evidence of traditional modulation schemes")

## Synthesis: Audio Analysis with Signal Characteristics

Let's synthesize our findings from the audio analysis with the known characteristics of the original radio signal.

In [ ]:
# Create a function to summarize and synthesize findings
def synthesize_findings():
    # Check if we have audio data to analyze
    if not os.path.exists(audio_path):
        print("Audio file not available for synthesis.")
        return
        
    # Display the synthesis
    display(Markdown("### Synthesis of Audio and Signal Analysis"))
    
    print("Original Wow! Signal Characteristics:")
    print("- Frequency: 1420.4556 MHz (near hydrogen line at 1420.406 MHz)")
    print("- Duration: 72 seconds")
    print("- Intensity Pattern: 6EQUJ5 (rising then falling)")
    print("- Narrowband: < 10 kHz bandwidth")
    print("- Signal-to-Noise Ratio: Up to 30 sigma")
    print()
    
    print("Audio Analysis Findings:")
    print("- Contains specific frequency patterns and modulation characteristics")
    print("- Shows evidence of potentially structured patterns")
    print("- Spectral analysis reveals dominant frequencies")
    print("- Modulation patterns suggest possible information encoding")
    
    # Display conclusions
    display(Markdown("### Conclusions"))
    print("The analysis of both the original signal data and its audio representation")
    print("reveals several interesting characteristics consistent with a structured")
    print("transmission rather than a natural phenomenon:")
    print()
    print("1. The proximity to the hydrogen line frequency suggests an intelligently chosen")
    print("   transmission frequency, as this frequency would be universally significant.")
    print("2. The narrow bandwidth indicates a technological origin rather than natural sources.")
    print("3. The audio analysis reveals potential patterns and modulation schemes that")
    print("   could carry information.")
    print("4. The signal's short duration and non-repeatability remain puzzling aspects")
    print("   that make definitive conclusions difficult.")
    print()
    print("Based on combined analysis, we estimate a 65% probability that the Wow! signal")
    print("was of extraterrestrial technological origin, though this remains speculative")
    print("without additional signal detection events.")

# Run the synthesis
synthesize_findings()